# Querying Protests Worldwide
Requirements:
* ArcGIS API for Python
`conda install -c esri arcgis`


In [1]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

In [89]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from arcgis.geometry.filters import intersects
from arcgis.mapping import generate_classbreaks
from datetime import datetime
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from georapid.formats import OutFormat
from georapid.protests import aggregate

## Creating a client
---
**NOTE**

Ensure that `os.environ['x_rapidapi_key']` holds your Rapid API Key!

---

In [3]:
host = 'geoprotests.p.rapidapi.com'
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

## Connecting to ArcGIS Online

In [4]:
gis = GIS()

In [86]:
def generate_aggregated_renderer(spatial_df, column='count'):
    alpha = 0.35
    class_breaks_renderer = generate_classbreaks(spatial_df, geometry_type='Polygon', field=column, method='esriClassifyNaturalBreaks', class_count=5, colors='YlOrRd', alpha=0.35)
    class_breaks_renderer['defaultSymbol']['color'][3] = int(alpha * 255)
    for class_break_info in class_breaks_renderer['classBreakInfos']:
        class_break_info['symbol']['color'][3] = int(alpha * 255)
    return class_breaks_renderer

def plot_aggregated(map_view, spatial_df, column='count'):
    """
    Plots the spatial dataframe as classified polygons using the specified map view.
    """
    if spatial_df.empty:
        print("The dataframe is empty!")
    else:
        class_breaks_renderer = generate_aggregated_renderer(spatial_df)
        drawing_info = {
            'renderer': class_breaks_renderer
        }
        feature_collection = spatial_df.spatial.to_feature_collection(drawing_info=drawing_info)
        map_view.add_layer(feature_collection)
        
def query_world_countries(gis, two_digit_country_code):
    """
    Returns the geographic feature of a country.
    """
    world_countries_item_id = '2b93b06dc0dc4e809d3c8db5cb96ba69'
    world_countries_item = gis.content.get(world_countries_item_id)
    world_countries_layer = world_countries_item.layers[0]
    return world_countries_layer.query(where=f"ISO='{two_digit_country_code}'")

## Query the protests and try to narrow down those related to the French pension reform strikes

In [59]:
protest_map = gis.map('France')
protest_map.basemap = 'osm'
protest_map

MapView(layout=Layout(height='400px', width='100%'))

In [10]:
date_of_interest = datetime(2023, 1, 19)

In [14]:
aggregated_protests_dict = aggregate(client, date_of_interest, OutFormat.ESRI)
aggregated_protests_fset = FeatureSet.from_dict(aggregated_protests_dict)
aggregated_protests_sdf = aggregated_protests_fset.sdf
aggregated_protests_sdf.sort_values(by='count', ascending=False)

,OBJECTID,count,timestamp,Shape__Area,Shape__Length,SHAPE
18,38893,1028,2023-01-19,54126587736.526367,866025.403784,"{""rings"": [[[-8779178.09360229, -1337491.6572]..."
49,38924,366,2023-01-19,54126587736.527466,866025.403784,"{""rings"": [[[97582.2951882072, 6287508.3428], ..."
17,38892,204,2023-01-19,54126587736.527832,866025.403784,"{""rings"": [[[-8779178.09360229, -837491.657200..."
14,38889,72,2023-01-19,54126587736.527344,866025.403784,"{""rings"": [[[-8995684.4445484, -712491.6572000..."
29,38904,50,2023-01-19,54126587736.527344,866025.403784,"{""rings"": [[[-7913152.68981785, -1587491.6572]..."
...,...,...,...,...,...,...
65,38940,1,2023-01-19,54126587736.527344,866025.403784,"{""rings"": [[[1613126.75181098, 6412508.3428], ..."
71,38946,1,2023-01-19,54126587736.527344,866025.403784,"{""rings"": [[[3561683.91032596, 7037508.3428], ..."
45,38920,1,2023-01-19,54126587736.527435,866025.403784,"{""rings"": [[[-118924.055757903, 6412508.3428],..."
8,38883,1,2023-01-19,54126587736.526367,866025.403784,"{""rings"": [[[-9861709.84833284, 1537508.3428],..."


In [72]:
protest_map = gis.map('France')
protest_map.basemap = 'osm'
plot_aggregated(protest_map, aggregated_protests_sdf)
protest_map

MapView(layout=Layout(height='400px', width='100%'))

## Filter the news related to protests being located in France

In [87]:
france_country_fset = query_world_countries(gis, two_digit_country_code='FR')
france_country_sdf = france_country_fset.sdf
france_country_sdf

,FID,COUNTRY,ISO,COUNTRYAFF,AFF_ISO,Shape__Area,Shape__Length,SHAPE
0,79,France,FR,France,FR,1162358729551.715332,8568685.86359,"{""rings"": [[[198339.953048288, 5246743.3982777..."


In [101]:
france_aggregated_protests_sdf = aggregated_protests_sdf.spatial.join(france_country_sdf[['COUNTRY', 'SHAPE']])
#france_aggregated_protests_sdf = france_aggregated_protests_sdf[france_aggregated_protests_sdf['COUNTRY'].notnull()]
france_aggregated_protests_sdf

,OBJECTID,count,timestamp,Shape__Area,Shape__Length,SHAPE,index_right,COUNTRY


In [110]:
intersects_filter = intersects(france_country_sdf.SHAPE[0])
intersects_filter

{'geometry': {'rings': [[[198339.953048288, 5246743.3982777],
    [193541.084866465, 5253763.75964162],
    [189057.911316411, 5255233.90480734],
    [173627.036889952, 5259751.83385319],
    [165479.439347937, 5258912.37802024],
    [160949.742967772, 5251578.74699019],
    [157764.002134707, 5258975.65684195],
    [154765.509627901, 5264651.72648147],
    [151301.35960817, 5268733.44960017],
    [147498.337517816, 5269069.50761908],
    [130676.96605276, 5267891.84600212],
    [127329.24440729, 5271465.11036461],
    [125419.587668049, 5276227.82351597],
    [121153.016801139, 5279281.56043099],
    [99847.4640713008, 5283558.55288909],
    [91683.7430935687, 5286595.38680593],
    [79716.9067185632, 5290516.88007709],
    [77120.156484783, 5289716.13823942],
    [74337.1656394856, 5287522.95720562],
    [72574.1873778486, 5279303.70769265],
    [73672.2605304239, 5274164.52762773],
    [75990.6948844371, 5269638.30131201],
    [75867.2498298132, 5266524.39126092],
    [75197.3434698